In [28]:
import time 
import os      
import requests
import bs4
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
from datetime import datetime

# Functions used:

In [29]:
def load_page(url):
    data = requests.get(url)
    htmlCode = BeautifulSoup(data.content, 'html.parser')
    return htmlCode
def load_page_table(htmlCode):
    table = htmlCode.find('table', attrs={'class':'table'}).find('tbody')
    return table
def get_table_rows(table):
    rows = table.find_all('tr')
    return rows
def getNextPageUrl(htmlCode):
    basicUrl = 'https://www.ad.co.il'
    if(htmlCode.find('li',attrs={'class': 'nextPage'}) != None):
        urlQueries = htmlCode.find('li',attrs={'class': 'nextPage'}).a['href'].strip()
    else:
        return False
    if(urlQueries):
        return 'https://www.ad.co.il'+ htmlCode.find('li',attrs={'class': 'nextPage'}).a['href'].strip()
    else:
        return False

# Load data per area function:

In [70]:
#Loads data per area
def load_area_data(areaName,startYear,endYear,dealsPerMonth,rooms = 4):
    ## Get deals for current city - 10 deals for each year between startYear and endYear
    ## 
    Dates = []
    Areas = []
    Addresses = []
    Rooms = []
    Surfaces = []
    Floors = []
    Prices = []
    PPSQM = []
    BuildYear = []
    DealMonth = []
    DealYear = []
    for year in range(startYear, endYear+1):
        #https://www.ad.co.il/nadlanprice?year=2010,2020&area=rishon-lezion-area
        currentPageUrl = 'https://www.ad.co.il/nadlanprice?year='+str(year)+','+str(year)+'&area='+areaName
        currentPageHtml = load_page(currentPageUrl);
        currentTable = load_page_table(currentPageHtml)
        rows = get_table_rows(currentTable)
        for month in range(12,0,-1):
            dealsPerMonthCounter = 0
            while(dealsPerMonthCounter < dealsPerMonth):
                
                for row in rows:
                    date_time_str = row.select('td')[0].text
                    date_time_obj = datetime.strptime(date_time_str, '%d/%m/%Y')
                    if(date_time_obj.month == month):
                        dealsPerMonthCounter+=1
                        #append row 
                        Dates.append(row.select('td')[0].text)
                        Areas.append(areaName)
                        Addresses.append(row.select('td')[2].text)
                        Rooms.append(row.select('td')[3].text)
                        Surfaces.append(row.select('td')[4].text)
                        Floors.append(row.select('td')[5].text)
                        Prices.append(row.select('td')[6].text)
                        PPSQM.append(row.select('td')[7].text)
                        BuildYear.append(row.select('td')[8].text)
                        DealMonth.append(date_time_obj.month)
                        DealYear.append(date_time_obj.year)
                        if(dealsPerMonthCounter == dealsPerMonth):
                            break
                    else:
                        if(date_time_obj.month < month):
                            print('Found only ',dealsPerMonthCounter,' deals out of ',dealsPerMonth,' in month:' ,month,' year:',year)
                            dealsPerMonthCounter = dealsPerMonth
                            break
                if(dealsPerMonthCounter < dealsPerMonth):
                    #get next page rows
                    tempUrlRequest = getNextPageUrl(currentPageHtml)
                    if(tempUrlRequest):
                        currentPageUrl = getNextPageUrl(currentPageHtml)
                        currentPageHtml = load_page(currentPageUrl)
                        currentTable = load_page_table(currentPageHtml)
                        rows = get_table_rows(currentTable)
                    else:
                        dealsPerMonthCounter = dealsPerMonth
                        
                                           
    df_area_deals = pd.DataFrame({'DealYear':DealYear,'DealMonth': DealMonth,'Date': Dates,'Area':Areas,
                              'Address': Addresses,'Rooms':Rooms,'Surface':Surfaces,'Floor':Floors,
                              'Price':Prices,'PPSQM':PPSQM,'BuildYear': BuildYear})
    print(areaName,' Data Frame:')
    display(df_area_deals)
    return df_area_deals

    

# Load all data from Gush Dan to a csv file function:
## Areas in Gush Dan: 
##### Tel-aviv area
##### Holon bat-yam
##### Ramat-gan givatayim
##### Ramat-hasharon herzliya
##### Rishon-lezion area
##### Ness-ziona rehovot
##### Ramla lod
##### Petah-tikva area
##### Hod-hasharon area
##### Raanana kfar-saba

In [74]:
#loads all data to csv
def load_data_to_csv():
    areasList = ['tel-aviv-area','holon-bat-yam','ramat-gan-givatayim','ramat-hasharon-herzliya','rishon-lezion-area',
                 'ness-ziona-rehovot','ramla-lod','petah-tikva-area','hod-hasharon-area','raanana-kfar-saba']
    dealsPerMonth = 30
    endYear = 2020
    startYear = 2010
    Dates = []
    Areas = []
    Addresses = []
    Rooms = []
    Surfaces = []
    Floors = []
    Prices = []
    PPSQM = []
    BuildYear = []
    DealMonth = []
    DealYear = []
    fullDf= pd.DataFrame({'DealYear': [],'DealMonth': [],'Date': [],'Area':[],'Address': [],'Rooms':[],'Surface':[]
                                 ,'Floor':[],'Price':[],'PPSQM':[],'BuildYear': []})
    for area in areasList:
        print(area,' start')
        areaDf = load_area_data(area,startYear,endYear,dealsPerMonth)
        fullDf = pd.concat([fullDf,areaDf],ignore_index=True)
        print(area,' done')
        print(" ------------------ ")
    
    fileName = 'data-v' + time.strftime("%H-%M-%S-%d-%m-%y")
    fullDf.to_csv('data/'+fileName + '.csv', encoding = "ISO-8859-8")
    return fileName

# Load the file using the load_data_to_csv() function

In [75]:
fileName = load_data_to_csv()
print ('Finished - the data set is ready : data/'+ fileName +'.csv')

tel-aviv-area  start
Found only  26  deals out of  30  in month: 2  year: 2019
tel-aviv-area  Data Frame:


,DealYear,DealMonth,Date,Area,Address,Rooms,Surface,Floor,Price,PPSQM,BuildYear
0,2010,12,31/12/2010,tel-aviv-area,פינס 1,3,10,6,"4,918,469",,2011
1,2010,12,31/12/2010,tel-aviv-area,מלצ''ט 36,5,128,,"950,000","7,421",1999
2,2010,12,31/12/2010,tel-aviv-area,קרליבך 1,3,82,4,"2,096,120","25,562",2013
3,2010,12,31/12/2010,tel-aviv-area,קרליבך 1,3,91,1,"1,898,195","20,859",2013
4,2010,12,31/12/2010,tel-aviv-area,קרליבך 1,5,121,12,"3,146,500","26,004",2013
...,...,...,...,...,...,...,...,...,...,...,...
3951,2020,1,24/1/2020,tel-aviv-area,,4,91,,"5,000,000","54,945",2012
3952,2020,1,24/1/2020,tel-aviv-area,קהילת קנדה 35,3,50,1,"1,280,000","25,600",1960
3953,2020,1,19/1/2020,tel-aviv-area,,7,347,,"25,672,173","73,983",2022
3954,2020,1,19/1/2020,tel-aviv-area,ליליאן 8,2,69,5,"3,140,000","45,507",2022


tel-aviv-area  done
 ------------------ 
holon-bat-yam  start
Found only  18  deals out of  30  in month: 12  year: 2018
Found only  28  deals out of  30  in month: 12  year: 2019
Found only  20  deals out of  30  in month: 2  year: 2019
Found only  26  deals out of  30  in month: 6  year: 2020
holon-bat-yam  Data Frame:


,DealYear,DealMonth,Date,Area,Address,Rooms,Surface,Floor,Price,PPSQM,BuildYear
0,2010,12,31/12/2010,holon-bat-yam,דרך בן גוריון 53,2,48,,"895,000","18,645",1980
1,2010,12,31/12/2010,holon-bat-yam,הגאונים 14,2,45,4,"600,000","13,333",1981
2,2010,12,31/12/2010,holon-bat-yam,הלפר 42,3,64,2,"850,852","13,294",1970
3,2010,12,31/12/2010,holon-bat-yam,"אח""י דקר 1",3.5,85,1,"1,081,081","12,718",1980
4,2010,12,31/12/2010,holon-bat-yam,חנקין 56,2,34,1,"695,000","20,441",1990
...,...,...,...,...,...,...,...,...,...,...,...
3927,2020,1,30/1/2020,holon-bat-yam,אתרוג 5,1.5,40,1,"1,130,000","28,250",1960
3928,2020,1,25/1/2020,holon-bat-yam,התחיה 21,2.5,42,4,"1,300,000","30,952",1980
3929,2020,1,25/1/2020,holon-bat-yam,חנקין 101,2.5,43,1,"1,100,586","25,595",1960
3930,2020,1,25/1/2020,holon-bat-yam,התחייה 23,2.5,42,,"1,300,000","30,952",1980


holon-bat-yam  done
 ------------------ 
ramat-gan-givatayim  start
ramat-gan-givatayim  Data Frame:


...

petah-tikva-area  done
 ------------------ 
hod-hasharon-area  start
Found only  6  deals out of  30  in month: 12  year: 2010
Found only  6  deals out of  30  in month: 11  year: 2010
Found only  13  deals out of  30  in month: 10  year: 2010
Found only  17  deals out of  30  in month: 7  year: 2010
Found only  17  deals out of  30  in month: 6  year: 2010
Found only  10  deals out of  30  in month: 5  year: 2010
Found only  29  deals out of  30  in month: 4  year: 2010
Found only  18  deals out of  30  in month: 3  year: 2010
Found only  1  deals out of  30  in month: 2  year: 2010
Found only  24  deals out of  30  in month: 12  year: 2011
Found only  22  deals out of  30  in month: 6  year: 2011
Found only  19  deals out of  30  in month: 3  year: 2011
Found only  14  deals out of  30  in month: 2  year: 2011
Found only  17  deals out of  30  in month: 12  year: 2012
Found only  10  deals out of  30  in month: 11  year: 2012
Found only  26  deals out of  30  in month: 10  year: 2012

,DealYear,DealMonth,Date,Area,Address,Rooms,Surface,Floor,Price,PPSQM,BuildYear
0,2010,12,31/12/2010,hod-hasharon-area,הוד השרון,4,117,7,"1,687,299","14,421",2011
1,2010,12,31/12/2010,hod-hasharon-area,"זלמן שז""ר 5",4,96,3,"1,610,000","16,770",2002
2,2010,12,19/12/2010,hod-hasharon-area,שרת משה 39,2.5,36,2,"728,000","20,222",1980
3,2010,12,10/12/2010,hod-hasharon-area,הדרור 5,6,150,6,"4,000,000","26,666",2011
4,2010,12,10/12/2010,hod-hasharon-area,,6,150,,"4,000,000","26,666",2011
...,...,...,...,...,...,...,...,...,...,...,...
2064,2020,1,14/1/2020,hod-hasharon-area,,6,198,,"7,220,000","36,464",2002
2065,2020,1,03/1/2020,hod-hasharon-area,,4,94,,"2,230,000","23,723",2003
2066,2020,1,03/1/2020,hod-hasharon-area,גולדה מאיר 8,4,144,7,"2,550,000","17,708",2011
2067,2020,1,03/1/2020,hod-hasharon-area,,4,144,,"2,550,000","17,708",2011


hod-hasharon-area  done
 ------------------ 
raanana-kfar-saba  start
Found only  29  deals out of  30  in month: 12  year: 2010
Found only  28  deals out of  30  in month: 2  year: 2010
Found only  25  deals out of  30  in month: 12  year: 2012
Found only  24  deals out of  30  in month: 6  year: 2012
Found only  16  deals out of  30  in month: 12  year: 2013
Found only  28  deals out of  30  in month: 6  year: 2013
Found only  22  deals out of  30  in month: 11  year: 2014
Found only  29  deals out of  30  in month: 12  year: 2016
Found only  21  deals out of  30  in month: 9  year: 2016
Found only  23  deals out of  30  in month: 7  year: 2016
Found only  12  deals out of  30  in month: 12  year: 2017
Found only  26  deals out of  30  in month: 10  year: 2017
Found only  28  deals out of  30  in month: 8  year: 2017
Found only  10  deals out of  30  in month: 2  year: 2017
Found only  24  deals out of  30  in month: 12  year: 2018
Found only  26  deals out of  30  in month: 11  year

,DealYear,DealMonth,Date,Area,Address,Rooms,Surface,Floor,Price,PPSQM,BuildYear
0,2010,12,31/12/2010,raanana-kfar-saba,פרדס משותף 27,4,120,1,"5,236,907","43,640",1960
1,2010,12,31/12/2010,raanana-kfar-saba,"שוהם צ""י 22",4.5,94,1,"1,850,000","19,680",1995
2,2010,12,31/12/2010,raanana-kfar-saba,"גבעון צ""י 32",5,125,,"2,780,000","22,240",1996
3,2010,12,31/12/2010,raanana-kfar-saba,ויצמן 131,3,60,2,"1,017,500","16,958",1973
4,2010,12,31/12/2010,raanana-kfar-saba,אוסטרובסקי 37,5,110,3,"1,906,000","17,327",1980
...,...,...,...,...,...,...,...,...,...,...,...
3680,2020,1,10/1/2020,raanana-kfar-saba,בית אל 90,5,173,,"3,200,000","18,497",1997
3681,2020,1,10/1/2020,raanana-kfar-saba,,6,163,,"2,570,000","15,766",2010
3682,2020,1,10/1/2020,raanana-kfar-saba,נחל ערוגות 1,6,163,,"2,570,000","15,766",2010
3683,2020,1,10/1/2020,raanana-kfar-saba,,5,181,,"2,840,000","15,690",2013


raanana-kfar-saba  done
 ------------------ 
Finished - the data set is ready : data/data-v21-18-50-25-06-22.csv


# The full Data Frame of Gush Dan:

In [77]:
data_file_name = 'data-v21-18-50-25-06-22'
fullDf = pd.read_csv('data/'+ data_file_name +'.csv', encoding= 'ISO-8859-8')
print('The full Data Frame of Gush Dan:')
fullDf

The full Data Frame of Gush Dan:


,Unnamed: 0,DealYear,DealMonth,Date,Area,Address,Rooms,Surface,Floor,Price,PPSQM,BuildYear
0,0,2010.0,12.0,31/12/2010,tel-aviv-area,פינס 1,3.0,10,6.0,"4,918,469",NaN,2011.0
1,1,2010.0,12.0,31/12/2010,tel-aviv-area,מלצ''ט 36,5.0,128,NaN,"950,000","7,421",1999.0
2,2,2010.0,12.0,31/12/2010,tel-aviv-area,קרליבך 1,3.0,82,4.0,"2,096,120","25,562",2013.0
3,3,2010.0,12.0,31/12/2010,tel-aviv-area,קרליבך 1,3.0,91,1.0,"1,898,195","20,859",2013.0
4,4,2010.0,12.0,31/12/2010,tel-aviv-area,קרליבך 1,5.0,121,12.0,"3,146,500","26,004",2013.0
...,...,...,...,...,...,...,...,...,...,...,...,...
36416,36416,2020.0,1.0,10/1/2020,raanana-kfar-saba,בית אל 90,5.0,173,NaN,"3,200,000","18,497",1997.0
36417,36417,2020.0,1.0,10/1/2020,raanana-kfar-saba,NaN,6.0,163,NaN,"2,570,000","15,766",2010.0
36418,36418,2020.0,1.0,10/1/2020,raanana-kfar-saba,נחל ערוגות 1,6.0,163,NaN,"2,570,000","15,766",2010.0
36419,36419,2020.0,1.0,10/1/2020,raanana-kfar-saba,NaN,5.0,181,NaN,"2,840,000","15,690",2013.0


# 400,631 kb of data extracted from ad.co.il